**Curso: Métodos Numéricos II  
Professor: Creto Augusto Vidal  
Semestre: 2020.1  
Equipe: DANIEL MAGALHÃES NUNES, ANTONIO AUGUSTO DA SILVA HOLANDA  
Tarefa 13**

**Com a matriz A abaixo, faça o que se pede:**
$$
A = \begin{bmatrix}
40 & 8 & 4 & 2 & 1\\ 
8 &30  &12  &6  & 2 \\ 
4 & 12 & 20 & 1 & 2\\ 
2 & 6 & 1 & 25 & 4\\ 
1 & 2 & 2 & 4 & 5
\end{bmatrix}
$$

**1) Implemente o método de Householder e aplique-o sobre A para encontrar:  
    i. a matriz tridiagonal <SPAN STYLE="text-decoration:overline">A</SPAN>  
    ii. a matriz acumulada H = H<sub>1</sub>H<sub>2</sub>H<sub>3</sub>  
2) Use os métodos da potência para encontrar os autovalores e autovetores da matriz <SPAN STYLE="text-decoration:overline">A</SPAN>.   
3) Usando a matriz H e os autovetores da matriz <SPAN STYLE="text-decoration:overline">A</SPAN> encontre os autovetores da matriz A.  
4) Encontre os autovalores da matriz A**

**Algoritimo do Método da Potência Regular**

In [2]:
import numpy as np

def metodoDaPotenciaRegular(A, v0, epsilon = 1e-4, max_iter = 10000):
    '''
    Esta função tenta encontrar o autovalor dominante e o autovetor correspondente da matriz A,
    usando o método de potência regular com os parâmetros:
     - A: (matriz) Matriz quadrada(nxn) que se deseja encontrar o autovalor dominante e o autovetor correspondente
     - v0: (vetor) Vetor arbitrário de tamanha n = dimensão da matriz A(nxn)
     - epsilon: (float) Critério de parada do método de potência quando alterações na solução são insignificantes
     - max_iter: (int) Critério de parada definitiva, quando o metodo não encontra uma solução aceitável
    '''
    ep = 0 # step 2
    vk = np.copy(v0) # step 3
    cont = 0

    while True:
        ep_velho = ep # step 4
        vk_velho = np.copy(vk) # step 5

        x1_velho = vk_velho / np.linalg.norm(vk_velho) # step 6
        vk = A.dot(x1_velho) # step 7
        ep = np.dot(x1_velho.T, vk) # step 8

        if np.abs((ep - ep_velho)/ep) < epsilon or cont > max_iter: # step 9
            break
        cont += 1
    return ep[0][0], vk.T # step 10

**Algoritimo do Método da Potência Inverso 2**

In [3]:
def metodoDaPotenciaInverso2(A, v0, epsilon = 1e-4, max_iter = 10000):
    ep = 0 # step 2
    vk = np.copy(v0) # step 3
    cont = 0

    while True:
        ep_velho = ep # step 4
        vk_velho = np.copy(vk) # step 5

        x1_velho = vk_velho / np.linalg.norm(vk_velho) # step 6
        vk = np.dot(np.linalg.inv(A), x1_velho) # step 7
        ep = np.dot(x1_velho.T, vk) # step 8

        if np.abs((ep - ep_velho)/ep) < epsilon or cont > max_iter: # step 9
            break
        cont += 1
    ep_menor = 1/ep[0][0]
    return ep_menor, vk.T # step 10

**Algoritimo do Método da Potência com Deslocamento**

In [4]:
def metodoDaPotenciaComDeslocamento(A, v0, mu):
    A_chapeu = A - mu*np.eye(A.shape[0])
    epc, xc = metodoDaPotenciaInverso2(A_chapeu, v0)
    ep = epc + mu
    x = xc
    return ep, x

**Algoritimo do Método da Potência Completo**

In [5]:
def metodoDaPotenciaCompleto(A, v0, epsilon = 1e-4, max_iter = 10000):

    if A.shape[0] == 1:
        return A, 1
    elif A.shape[0] == 2:
        ep_maior, v_maior = metodoDaPotenciaRegular(A, v0)
        ep_menor, v_menor = metodoDaPotenciaInverso2(A, v0)
        return [ep_menor, ep_maior], [v_menor, v_maior]
    else:
        ep_maior, v_maior = metodoDaPotenciaRegular(A, v0)
        ep_menor, v_menor = metodoDaPotenciaInverso2(A, v0) 
        autovalores = [ep_menor, ep_maior]
        autovetores = [v_menor, v_maior]
        passo = (ep_maior - ep_menor) / A.shape[0]
        mu = passo
        
        while len(set(autovalores)) < A.shape[0]:
            ep, vk = metodoDaPotenciaComDeslocamento(A, v0, mu)
            autovalores.append(ep)
            autovetores.append(vk)
            mu += passo
        
        return np.array(autovalores), np.array(autovetores).reshape(A.shape)

**Algoritimo do Método Householder**

In [27]:
def matrizHouseholder(A, i):
    n = A.shape[0]
    I = np.eye(n)
    w = np.zeros(n).reshape(1,n)
    w_linha = np.zeros(n).reshape(1,n)
    
    w[0, i+1:] = A[i+1:, i]
    lw = np.linalg.norm(w)
    w_linha[0, i+1] = lw
    N = w - w_linha
    n = N / np.linalg.norm(N)

    H = I - 2*np.dot(n.T, n)

    return H


def householder(A):
    n = A.shape[0]
    H = np.eye(n)
    H_i = 0
    A_i = 0
    A_barra = 0
    A_i1 = np.copy(A)
    for i in range(n-2):
        H_i = matrizHouseholder(A_i1, i)

        A_i = np.dot(np.dot(H_i.T, A_i1), H_i)

        A_i1 = np.copy(A_i)

        H = np.dot(H, H_i)

    A_barra = np.copy(A_i)

    return A_barra, H

**Encontrando os autovalores e os autovetores da matriz A**

In [20]:
A = np.array([[40, 8,  4,  2, 1], # Definindo matriz A
              [8, 30, 12,  6, 2],
              [4, 12, 20,  1, 2],
              [2,  6,  1, 25, 4],
              [1,  2,  2,  4, 5]])


A_barra, H = householder(A) # Chamando do método householder
v0 = np.array([1, 0, 0, 0, 0]).reshape(5, 1) # Definindo vetor v0
ep, vk = metodoDaPotenciaCompleto(A_barra, v0) # Chamando do método da potência

In [28]:
# Autovalores de A
ep

array([ 4.01490122, 49.38110001, 11.64248171, 23.64910867, 23.64719504])

In [29]:
# Autovetores de A
autovetores = np.dot(H, vk)
np.round(autovetores, 3)

array([[ 2.0000e-03, -9.0000e-03,  7.3000e-02, -1.2200e-01,  2.0400e-01],
       [ 3.0116e+01,  3.0256e+01,  3.5900e+00,  8.5100e-01,  2.0600e-01],
       [ 1.5044e+01,  1.5137e+01,  1.9080e+00,  1.2100e-01, -1.1900e-01],
       [ 7.5620e+00,  7.4900e+00,  1.2360e+00,  2.4800e-01, -5.2000e-02],
       [ 3.7470e+00,  3.8120e+00,  3.7000e-01, -7.0000e-03, -1.2000e-02]])